# Import the Libraries

In [ ]:
import pandas as pd
import unidecode

# Run the code to get player data in list

In [ ]:
# alphabet = 'abcdefghijklmnopqrstuvwxyz'
# base_url = 'https://www.basketball-reference.com/players'
# temp = []

# for letter in alphabet:
#     df = pd.read_html(f"{base_url}/{letter}/")[0]
#     df['Letter'] = [letter]*len(df)
#     temp.append(df)
    
# data = pd.concat(temp)
# data = data.reset_index(drop = True)

# data.to_csv('all_players.csv', index = False)

# Get data from csv file instead

In [ ]:
data = pd.read_csv('all_players.csv')
data

# Get stars and middleNames

In [ ]:
starPlayers = [player for player in data.Player if player[-1] == '*']
players = [unidecode.unidecode(player) if player not in starPlayers else unidecode.unidecode(player[0:-1]) for player in data.Player]
data.Player = players
middleNames = [name for name in data.Player if (len(name.split(' '))>2)]

# Set up loop to get player dataframes

In [ ]:
base_url = 'https://www.basketball-reference.com/players'
CornerCases={
    "Luc Mbah a Moute":  "mbahalu",
    "Metta World Peace": "artesro",
    "Logan Vander Velden": "vandelo",
    "Nene": "hilarne",
    "Jeff Ayres": "pendeje",
    "Mark Baker": "bakerla",
    "J.J. Barea": "bareajo",
    "George Bon Salle": "bonsage", ## Could check for bon with the vans and vons
    "Clint Capela": "capelca",
    "Dick Clark": "clarkri",
    "Gigi Datome": "datomlu",
    "Vinny Del Negro": "delnevi",
    "Nando De Colo": "decolna",  ## and dels
    "Rich Eichhorst": "eichhdi",
    "Wayne Engelstad": "englewa",
    "Bill Franklin": "frankwi",
    "Enes Freedom": "kanteen",
    "Smokey Gaines": "gaineda",
    "Gene Gallette": "gillege",
    "Bubbles Hawkins": "hawkiro",
    "Walt Hazzard": "abdulma",
    "Paul Hogue": "hoguedu",
    "Alfredrick Hughes": "hugheri",
    "Howie Jolliff": "jolifho",
    "Derrick Jones Jr.": "jonesde",
    "Jaren Jackson Jr.": "jacksja",
    "Maxi Kleber": "klebima",
    "Didi Louzada": "louzama",
    "Sheldon Mac": "mcclesh",
    "D.J. Mbenga": "mbengdj",
    "Frank Ntilikina": "ntilila",
    "Cedi Osman": "osmande",
    "Sasha Pavlovic": "pavloal",
    "Jim Phelan": "phelaja",
    "Don Putman": "putnado",
    "Bob Rensberger": "rensbro",
    "Mouhamed Sene": "senesa",
    "Edy Tavares": "tavarwa",
    "Gene Tormohlen": "tormobu",
    "Marcus Vinicius": "vincima",
    "Henry Walker": "walkebi",
    "Mo Williams": "willima",
    "Jimmy Foster": "fosteja"
    
}
extensions = []


for i in range(len(data)):
    
    player = data.iloc[i].Player
    FullName = unidecode.unidecode(player)
    
    if FullName in CornerCases.keys():
        extension = CornerCases[FullName]
        data.at[i,'Letter'] = extension[0]
    else:
        
        FullName = FullName.replace("'","")
        FullName = FullName.replace(".","")
        FullName = FullName.replace("-","")
        letter = data.iloc[i].Letter
        name_array = FullName.split(' ')
        
        if len(name_array)>2:
            first2 = name_array[0][0:2].lower()
            

            if name_array[1].lower() == "von" or (name_array[1].lower()=="van"):
                first5 = name_array[1][0:3].lower() + name_array[2][0:2].lower()
            else:
                for j in range(len(name_array)-1, 0, -1):
                    if name_array[j][0].lower() == letter:
                        first5= name_array[j][0:5].lower()
                        break

        else:
            first5 = name_array[1][0:5].lower()
            first2 = name_array[0][0:2].lower()

        extension = first5 + first2

    extensions.append(extension)

data['Extension'] = extensions    

In [ ]:
print("FullName: ", FullName)
print("extension: ", extension)
print("letter: ", letter)
print("Possible url: ", f'{base_url}/{letter}/{extension}01.html')

# Check duplicate extensions

In [ ]:
########Put logic to get columns for number#############
data = data.sort_values(['Extension','From']).reset_index(drop = True)

CornerCaseNumbers = {
    "hairspj01": 2,
    "munfoxa01": 2,
    "brownch03": 5,
    "brownke02": 3,
    "burtode01": 2,
    "davisjo03": 6,
    "fostemi01": 2,
    "greenja03": 5,
    "howarma01": 2,
    "jacksjo01": 2,
    "johnsda05": 8,
    "johnsja03": 5,
    "johnske05": 7,
    "jonesca02": 3,
    "jonesja03": 4,
    "jonesma04": 5,
    "smithja03": 4,
    "smithto01": 2,
    "tilliki01": 2,
    "washity01": 2,
    "willial04": 6,
    "willibr02": 3,
    "willija05": 6,
    "willima06": 7,
    "williro03": 4,
    
    
}

resetNum = 1
prevNum = 1
numbers = [1]
currentNum = 1

for i in range(1,len(data)):
    if data.iloc[i].Extension == data.iloc[i-1].Extension:
        currentNum = prevNum + 1
    else: 
        currentNum = resetNum
    
    key = f"{data.iloc[i].Extension}{str(currentNum).zfill(2)}"
    if key in CornerCaseNumbers.keys(): 
        currentNum = CornerCaseNumbers[key]
    
    prevNum = currentNum
    numbers.append(currentNum)
    
numbers = [str(number).zfill(2) for number in numbers]
data['Number'] = numbers

# Generate the correct urls (and verify no duplicates)

In [ ]:
urls = []

for i in range(len(data)):
    row = data.iloc[i]
    url = f"{base_url}/{row.Letter}/{row.Extension}{row.Number}.html"
    urls.append(url)
    
data['url'] = urls
assert(len(data['url'].unique()) == len(data))

# Read htmls and write csvs

In [ ]:
import time
import os

len_data = len(data)
bad_rows = []

raw_dir = 'raw_player_dfs/'
if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)

for i in range(len_data):
    
    row = data.iloc[i]
    path = f'raw_player_dfs/{row.Letter}'
    
    if not os.path.exists(path):
        os.makedirs(path)
    file = f"{path}/{row.Extension}{row.Number}.csv"

    if os.path.isfile(file) and row.To != 2023: ####PUT CONDITION FOR CURRENT PLAYERS
        continue
    else:
        try:
            time.sleep(2)
            print(f"{i+1}/{len_data}", row.Player)
            player_df = pd.read_html(row.url)[0]
            player_df.to_csv(file, index = False)
        except:
            print(f'Did not work at index {i}:', row.Player)
            bad_rows.append(row)
    
            
try:
    bad_df = pd.concat(bad_rows,axis =1).T
except:
    print("All players generated! Yay!")

# Clean up dataframe

In [ ]:
import os

alphabet = 'abcdefghijklmnopqrstuvwxyz'
ALPHABET = alphabet.upper()

clean_dir = 'player_dfs/'
if not os.path.exists(clean_dir):
    os.makedirs(clean_dir)
    
len_data = len(data)
bad_rows = []
    
for index,row in data.iterrows():
    
    print(f'{index+1}/{len_data}',row.Player)
    
    try:
        path = f'{clean_dir}{row.Letter}'
        if not os.path.exists(path):
            os.makedirs(path)
    #####################################READ IN RAW PLAYER DF####################################
        file = f'{row.Letter}/{row.Extension}{row.Number}.csv'
        raw_player_df = pd.read_csv(f'raw_player_dfs/{file}')
        player_df = raw_player_df.dropna(subset = ['Pos'])

    ####################################CHECK FOR DNP SEASONS################################
        if player_df.G.dtype == 'object':
            DNP = [False if (row[1].G[0] in alphabet) or (row[1].G[0] in ALPHABET) else True for row in player_df.iterrows()]
            player_df = player_df[DNP]

    ########################DROP DUPLICATE SEASON TO GET TOT#####################################
        player_df = player_df.drop_duplicates(subset = ['Season'], keep = 'first')
        player_df = player_df.reset_index(drop = True)

    ############################WRITE THE CSV FILE#######################################
        file = f'{clean_dir}{file}'
        player_df = player_df.set_index('Season')
        player_df['Player'] = [row.Player]*len(player_df)
        player_df['Pos'] = [row.Pos]*len(player_df)
        player_df['Ht'] = [row.Ht]*len(player_df)
        player_df.to_csv(file)
    
    except:
        print(f'Did not work at index {i}:', row.Player)
        bad_rows.append(row)
        
try:
    bad_df = pd.concat(bad_rows,axis =1).T
except:
    print("All players generated! Yay!")

# Generate ratings for each season

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np    


seasons = list(range(1980,2024))
target_num_players = 250
weightings = {"PTS": 5,"AST": 3.5,"TRB": 3,"STL": 1.5,"BLK": 1.5,"eFG%": 2,"FT%": 1,"TOV": -1,"PF": -1}
stats = list(weightings.keys())


season_dir = 'season_ratings/'
if not os.path.exists(season_dir):
    os.makedirs(season_dir)


for i in range (len(seasons)):
    
    season_data = []
    ###############INSERT LOOOOOOOOOOOOOOP HERE########################
    season = seasons[i]
    season_string = f"{season-1}-{str(season%100).zfill(2)}"
    print(season_string)

    #######################GET ELIGIBLE PLAYERS########################
    players = data.query(f"From <= {season} and To >= {season}")

    for j in range(len(players)):
        row = players.iloc[j]
        player_data = pd.read_csv(f"player_dfs/{row.Letter}/{row.Extension}{row.Number}.csv", index_col = 'Season') 
        if season_string in player_data.index:
            season_row = player_data.loc[season_string]
            season_data.append(season_row)

    season_data = pd.concat(season_data,axis =1).T
    season_data = season_data.set_index("Player")
    season_data = season_data.fillna(0)
    if len(season_data) > target_num_players:
        min_games = int(season_data.sort_values('G', ascending = False).iloc[target_num_players].G)
        season_data = season_data.query(f"G >= {min_games}")

    scaler = StandardScaler()
    season_data[stats] = scaler.fit_transform(np.array(season_data[stats]))
    season_data = season_data.filter(['Player','Pos','Ht']+ stats)
    season_data[stats] = season_data[stats].multiply(list(weightings.values()))
    season_data['ChrissyMetric'] = season_data[stats].sum(axis = 1)
    season_data = season_data.sort_values("ChrissyMetric", ascending = False) 
    
    path = f'{season_dir}{season_string}.csv'
    season_data.to_csv(path)

In [ ]:
plt.hist(season_data.ChrissyMetric)

# DEBUG 

In [ ]:
index = 1256 ##### GETTING THE PLAYER (DOESNT MATTER) (1256 is Anthony Edwards)
season = 2022 ##### THIS IS IMPORTANT

row = data.iloc[index]

###############GET THE SEASONAL ROW FOR THIS PLAYER##############
season_string = f"{season-1}-{season%100}"
season_row = pd.read_csv(f"player_dfs/{row.Letter}/{row.Extension}{row.Number}.csv", index_col = 'Season').loc[season_string]